In [70]:
# %load cs_snf.py
""" Implementation of Similarity Network Fusion (SNF) based similarity combination
    [1] Zhang, Yong, Xiaohua Hu, and Xingpeng Jiang. "Multi-view clustering of microbiome samples by robust similarity network fusion and spectral clustering." IEEE/ACM transactions on computational biology and bioinformatics 14.2 (2015): 264-271.
    [2] Olayan, Rawan S., Haitham Ashoor, and Vladimir B. Bajic. "DDR: efficient computational method to predict drug–target interactions using graph mining and machine learning approaches." Bioinformatics 34.7 (2018): 1164-1173.
"""
import numpy as np


from combine_sims import Combine_Sims_Ave
from mv_model_trans.SNF import SNF


class Combine_Sims_SNF(Combine_Sims_Ave):
    """ !!! SNF is used for transductive model only"""
    def __init__(self, k=3, num_iters=2, alpha = 1):
           super().__init__()
           self.k = k #the number of neareast neighbours
           self.num_iters = num_iters # the number of iterations
           self.alpha = alpha # paramter for normalized function
           
           self.copyable_attrs=self.copyable_attrs+['k','num_iters','alpha']
    #----------------------------------------------------------------------------------------     
    
    def combine(self, Ss, Y):
        if Ss.shape[0] == 1:
            S = Ss[0]
        else:
            S = SNF(Ss, self.k, self.num_iters, self.alpha)
        w = np.zeros(Ss.shape[0])
        return S, w
    #----------------------------------------------------------------------------------------

In [71]:
S1 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_ddi.txt")
S2 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_disease.txt")
S3 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_se.txt")
S4 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_tanimoto.txt")
Y_ = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt").T
similarity_matrices = np.array([S1, S2, S3, S4])

In [72]:
com_snf = Combine_Sims_SNF(k=3, num_iters=2, alpha = 1)


In [79]:
S_com_snf_d,w = com_snf.combine(similarity_matrices, Y_)

In [80]:

np.savetxt('S_com_snf_d.txt', S_com_snf_d, delimiter='\t')

In [81]:
S2_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_disease.txt")
S1_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_ppi.txt")
S3_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_sw-n.txt")

Y_1 = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p])

S_com_snf_p,w =com_snf.combine(similarity_matrices_p,Y_1)

In [82]:
np.savetxt('S_com_snf_p.txt', S_com_snf_p, delimiter='\t')

# HSIC

In [38]:
# %load hsic.py
import numpy as np
from scipy.optimize import minimize,Bounds
from combine_sims import Combine_Sims_Ave


class HSIC(Combine_Sims_Ave):
    """ 
    Impelementation of Hilbert–Schmidt independence criterion-based multiple similarities fusion based on
    matlab codes of [1] which is availble at https://figshare.com/s/f664b36119c60e7f6f30
    [1] Ding, Yijie, Jijun Tang, and Fei Guo. "Identification of drug–target interactions via 
    dual laplacian regularized least squares with multiple kernel fusion." Knowledge-Based Systems 204 (2020): 106254.
    """
    def __init__(self, v1=2**-1, v2=2**-4, seed=0):
        super().__init__()
        self.v1 = v1
        self.v2 = v2
        self.seed = seed
        self.copyable_attrs=self.copyable_attrs+['v1','v2','seed']
        """
        v1 = [2**0, 2**-1, ..., 2**-5]
        v2 = [2**0, 2**-1, ..., 2**-5]
        """
    #----------------------------------------------------------------------------------------     
        
    def _compute_weights(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        n = Ss.shape[1] # the number of rows in Ss[0]
        Ss1 = np.zeros(Ss.shape)
        for i in range(self._num_sims):
            Ss1[i] = self._process_sim(Ss[i])
        S_ideal = Y@Y.T # U in paper
        S_ideal = self._normalize_sim(S_ideal)
        
        H = np.eye(n)-np.ones(Ss1[0].shape, dtype=float)/n
        M = np.zeros((self._num_sims,self._num_sims), dtype=float) # the similarity between input similarity matrices
        for i in range(self._num_sims):
            for j in range(i,self._num_sims):
                mm = self._alignment(Ss1[i],Ss1[j])
                m1 = self._alignment(Ss1[i],Ss1[i])
                m2 = self._alignment(Ss1[j],Ss1[j])
                ss = mm/(np.sqrt(m1)*np.sqrt(m2))
                M[i,j] = M[j,i] = ss
        d1 = np.sum(M, axis=1)
        D1 = np.diag(d1)
        LapM = D1-M
        
        a = np.zeros(self._num_sims)
        for i in range(self._num_sims):
            kk = H@Ss1[i]@H
            aa = np.trace(kk.T@S_ideal)
            a[i] = (n**-2)*aa # n-1 in matlab code
        
        prng = np.random.RandomState(self.seed)
        w = prng.rand(self._num_sims)
        w = w/np.sum(w)
        bnds = Bounds(np.zeros(self._num_sims),np.ones(self._num_sims)) # eq.14c
        cons = ({'type': 'eq', "fun": self._constraint_eq }) # eq.14d
        res = minimize(self._f_obj, w, args=(a,LapM), method='SLSQP', bounds=bnds, constraints=cons)
        w = res.x
        return w
    #----------------------------------------------------------------------------------------  
    def _process_sim(self, S):
        # make similarity matrix symmetric
        S1 = (S+S.T)/2 
        # make similarity matrix PSD
        eig_values = np.linalg.eigvals(S)
        eig_values = np.real_if_close(eig_values) # keep the real part of eig_values
        ev_min = np.min(eig_values)
        e = max(0.0, -1.0*ev_min+1e-4)
        e1 = e.real
        S1 = S1 + e1*np.eye(S.shape[0])
        
        S1 = self._normalize_sim(S1)
        return S1
    
    def _normalize_sim(self, S):
        min_nz = np.min(S[np.nonzero(S)]) # the mininal none zero value
        S[S==0] = min_nz
        D = np.diag(S)
        D = np.sqrt(D)
        S1 = S/(np.outer(D,D)) 
        return S1
    #----------------------------------------------------------------------------------------  
    
    def _alignment(self, S1, S2):
        # same with np.trace(S1.T@S2)
        A = S1*S2
        a = A.sum()
        return a     
    #----------------------------------------------------------------------------------------  
    
    def _f_obj(self, w, a, LapM):
        #  eq.14a.
        J = -1*w@a + self.v1*w.T@LapM@w +self.v2*np.linalg.norm(w,2)**2 # last term is equalient to w@w
        return J
    #----------------------------------------------------------------------------------------
    
    def _constraint_eq(self, w):
        """
        return value must come back as 0 to be accepted 
        if return value is anything other than 0 it's rejectedas not a valid answer.
        """
        s = np.sum(w)-1
        return s
    #----------------------------------------------------------------------------------------

In [52]:
com_hsic = HSIC(v1=2**-1, v2=2**-4, seed=0)

S1 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_ddi.txt")
S2 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_disease.txt")
S3 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_se.txt")
S4 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_tanimoto.txt")
Y_ = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt").T
similarity_matrices = np.array([S1, S2, S3, S4])

com_hsic._compute_weights(similarity_matrices,Y_)


S_com_hsic_d = 0.2501135*S1+0.24991581*S2+0.24992715*S3+0.25004353*S4
np.savetxt('S_com_hsic_d.txt', S_com_hsic_d, delimiter='\t')
com_hsic._process_sim(S_com_hsic)

com_hsic._normalize_sim(S_com_hsic)

com_hsic._constraint_eq(com_hsic._compute_weights(similarity_matrices,Y_))

0.0

In [53]:
S2_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_disease.txt")
S1_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_ppi.txt")
S3_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_sw-n.txt")

Y_1 = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p])

com_hsic._compute_weights(similarity_matrices_p,Y_1)

S_com_hsic_p = 0.33333129*S2_p+0.33335728*S1_p+0.33331143*S3_p
np.savetxt('S_com_hsic_p.txt', S_com_hsic_p, delimiter='\t')

# LIC

In [33]:
# %load combine_sims.py
import numpy as np
from base.csbase import Combine_Sims_Base
from sklearn.neighbors import NearestNeighbors

# class Combine_Sims_base:
#     def __init__(self):
#         pass
# #---------------------------------------------------------------------------------------- 
        
#     def combine(self, Ss, Y=None):
#         self._num_sims = Ss.shape[0]
#         w = self._compute_weights()
#         S = np.average(Ss,axis=0,weights=w) # aggragate weights based on w
#         return S, w
    
#     def _compute_weights(self):
#         w = np.full(self._num_sims,1.0/self._num_sims, dtype=float) # the sum of weights could not be one.
#         return w

class Combine_Sims_Ave(Combine_Sims_Base):
    """ 
    using equal wieghts and the weight of each similairty is 1/num_sims
    A multiple kernel learning algorithm for drug-target interaction prediction  BMC Bioinf 16
    It aslo the base class of all Combine_Sims classes
    """
    def __init__(self):
        self.copyable_attrs  = []
#---------------------------------------------------------------------------------------- 
        
    def combine(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        w = self._compute_weights(Ss, Y)
        S = np.average(Ss,axis=0,weights=w) # aggragate similarity based on w
        return S, w
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y): # Ss and Y are not used in this function
        w = np.full(self._num_sims,1.0/self._num_sims, dtype=float) # the sum of weights could not be one.
        return w
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------          


class Combine_Sims_KA(Combine_Sims_Ave):
    """ 
    kernel alignment (KA) heuristics weighting method. 
    The weights of a similalrity is propotional to its aligment to the idal kernel
    A multiple kernel learning algorithm for drug-target interaction prediction  BMC Bioinf 16
    """
    def __init__(self):
        super().__init__()
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        n = Ss.shape[1] # the number of rows in Ss[0]
        w = np.zeros(self._num_sims, dtype=float) 
        S_ideal = Y@Y.T
        for i in range(self._num_sims):
            w[i] = self._alignment(Ss[i],S_ideal)/(n*np.sqrt(self._alignment(Ss[i],Ss[i])))
        w = w/np.sum(w)
        return w
#----------------------------------------------------------------------------------------     
        
    def _alignment(self, S1, S2):
        A = S1*S2
        a = A.sum()
        return a     
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------   

class Combine_Sims_KA2(Combine_Sims_KA):
    """ 
    Difference with Combine_Sims_KA: S only contain k largest values in each row excluding the diagonal elements 
    """
    def __init__(self, k=5):
        super().__init__()
        self.k = k
        self.copyable_attrs.append('k')
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        n = Ss.shape[1] # the number of rows in Ss[0]
        w = np.zeros(self._num_sims, dtype=float) 
        neigh = NearestNeighbors(n_neighbors=self.k, metric='precomputed')
        neigh.fit(np.zeros(Ss[0].shape))
                
        S_ideal = Y@Y.T
          
        for i in range(self._num_sims):
            S = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            knn = neigh.kneighbors(1-S, return_distance=False)
            S1 = np.zeros(S.shape)
            for j in range(S.shape[0]):
                jj = knn[j]
                S1[j, jj] = S[j, jj]
            w[i] = self._alignment(S1,S_ideal)/(n*np.sqrt(self._alignment(S1,S1)))
        w = w/np.sum(w)
        return w
#----------------------------------------------------------------------------------------     
        
    def _alignment(self, S1, S2):
        A = S1*S2
        a = A.sum()
        return a     
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------   
    
        
class Combine_Sims_Limb(Combine_Sims_Ave):
    """ 
    the weight is proprotional to the 1-local imbalance
    """
    def __init__(self, k=5):
        super().__init__()
        self.k = k
        self.copyable_attrs.append('k')    
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        w = np.zeros(self._num_sims, dtype=float) 
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            w[i] = 1-self._cal_limb(S1, Y, self.k)
        w = w/np.sum(w)
        return w 
#----------------------------------------------------------------------------------------         
        
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        knns = neigh.kneighbors(1 - S, return_distance=False)
        
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            for j in range(Y.shape[1]):
                if Y[i,j] == 1: # only consider "1" 
                    C[i,j] = k-np.sum(Y[ii,j])
        C = C/k
        milb = np.sum(C)/np.sum(Y)
        
        return milb
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------
        
    
class Combine_Sims_Limb2(Combine_Sims_Limb):
    """ 
    Difference with Combine_Sims_Limb
    the weight is proprotional to the 1/(local imbalance)
    """
    def __init__(self, k=5):
        super().__init__(k)
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        w = np.zeros(self._num_sims, dtype=float) 
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            w[i] = 1/self._cal_limb(S1, Y, self.k)
        w = w/np.sum(w)
        return w 
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------


class Combine_Sims_Limb3(Combine_Sims_Limb):
    """ 
    Difference with Combine_Sims_Limb
    considering the influence of similarities
    !!!!!! This is one finally used in the paper
    """
    def __init__(self, k=5):
        super().__init__(k)
        
#---------------------------------------------------------------------------------------- 
    
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        idx = np.where(S>1.0)
        knns = neigh.kneighbors(1 - S, return_distance=False)
                
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            s = S[i,ii]
            z = np.sum(s)
            if z == 0:
                z=1
            C[i] = 1-s@Y[ii,:]/z
        C *= Y
        milb = np.sum(C)/np.sum(Y)
        
        return milb
#---------------------------------------------------------------------------------------- 
#----------------------------------------------------------------------------------------
        

class Combine_Sims_LowestLimb(Combine_Sims_Limb):
    """ 
    choose the one similarity with lowerest local imbalance
    """
    def __init__(self, k=5):
        super().__init__(k)
#---------------------------------------------------------------------------------------- 
    
    def _compute_weights(self, Ss, Y):
        w = np.zeros(self._num_sims, dtype=float) 
        limb_ds = np.ones(self._num_sims, dtype=float)
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            limb_ds[i] = self._cal_limb(S1, Y, self.k)
        min_idx = np.argmin(limb_ds)
        w[min_idx] = 1.0
        # w = w/np.sum(w)
        return w 
#----------------------------------------------------------------------------------------

class Combine_Sims_Single(Combine_Sims_Ave):
    """ 
    Only use one single similarity
    """
    def __init__(self, ind=0):
        super().__init__()
        self.ind = ind # ind similarity is used, others' weights are set as 0
        self.copyable_attrs.append('ind')
#---------------------------------------------------------------------------------------- 
    
    def _compute_weights(self, Ss, Y):
        w = np.zeros(self._num_sims, dtype=float) 
        w[self.ind] = 1.0
        return w 
#----------------------------------------------------------------------------------------

#----------------------------------------------------------------------------------------

In [34]:
com_lic = Combine_Sims_Limb3(k=5)
S1 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_ddi.txt")
S2 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_disease.txt")
S3 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_se.txt")
S4 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_tanimoto.txt")
Y_ = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt").T
similarity_matrices = np.array([S1, S2, S3, S4])

In [35]:
com_lic._compute_weights(similarity_matrices,Y_)

array([0.2752286 , 0.16697413, 0.24368714, 0.31411014])

In [54]:
S_com_lic_d = 0.2752286*S1+0.16697413*S2+0.24368714*S3+0.31411014*S4


np.savetxt('S_com_lic_d.txt', S_com_lic_d, delimiter='\t')

In [56]:
S2_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_disease.txt")
S1_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_ppi.txt")
S3_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_sw-n.txt")

Y_1 = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p])

com_lic._compute_weights(similarity_matrices_p,Y_1)

array([0.19642949, 0.19249905, 0.61107146])

In [60]:
S_com_lic_p = 0.19642949*S2_p+0.19249905*S1_p+0.61107146*S3_p


np.savetxt('S_com_lic_p.txt', S_com_lic_p, delimiter='\t')

# KA

In [86]:
# %load combine_sims.py
import numpy as np
from base.csbase import Combine_Sims_Base
from sklearn.neighbors import NearestNeighbors

# class Combine_Sims_base:
#     def __init__(self):
#         pass
# #---------------------------------------------------------------------------------------- 
        
#     def combine(self, Ss, Y=None):
#         self._num_sims = Ss.shape[0]
#         w = self._compute_weights()
#         S = np.average(Ss,axis=0,weights=w) # aggragate weights based on w
#         return S, w
    
#     def _compute_weights(self):
#         w = np.full(self._num_sims,1.0/self._num_sims, dtype=float) # the sum of weights could not be one.
#         return w

class Combine_Sims_Ave(Combine_Sims_Base):
    """ 
    using equal wieghts and the weight of each similairty is 1/num_sims
    A multiple kernel learning algorithm for drug-target interaction prediction  BMC Bioinf 16
    It aslo the base class of all Combine_Sims classes
    """
    def __init__(self):
        self.copyable_attrs  = []
#---------------------------------------------------------------------------------------- 
        
    def combine(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        w = self._compute_weights(Ss, Y)
        S = np.average(Ss,axis=0,weights=w) # aggragate similarity based on w
        return S, w
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y): # Ss and Y are not used in this function
        w = np.full(self._num_sims,1.0/self._num_sims, dtype=float) # the sum of weights could not be one.
        return w
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------          


class Combine_Sims_KA(Combine_Sims_Ave):
    """ 
    kernel alignment (KA) heuristics weighting method. 
    The weights of a similalrity is propotional to its aligment to the idal kernel
    A multiple kernel learning algorithm for drug-target interaction prediction  BMC Bioinf 16
    """
    def __init__(self):
        super().__init__()
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        n = Ss.shape[1] # the number of rows in Ss[0]
        w = np.zeros(self._num_sims, dtype=float) 
        S_ideal = Y@Y.T
        for i in range(self._num_sims):
            w[i] = self._alignment(Ss[i],S_ideal)/(n*np.sqrt(self._alignment(Ss[i],Ss[i])))
        w = w/np.sum(w)
        return w
#----------------------------------------------------------------------------------------     
        
    def _alignment(self, S1, S2):
        A = S1*S2
        a = A.sum()
        return a     
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------   

class Combine_Sims_KA2(Combine_Sims_KA):
    """ 
    Difference with Combine_Sims_KA: S only contain k largest values in each row excluding the diagonal elements 
    """
    def __init__(self, k=5):
        super().__init__()
        self.k = k
        self.copyable_attrs.append('k')
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        n = Ss.shape[1] # the number of rows in Ss[0]
        w = np.zeros(self._num_sims, dtype=float) 
        neigh = NearestNeighbors(n_neighbors=self.k, metric='precomputed')
        neigh.fit(np.zeros(Ss[0].shape))
                
        S_ideal = Y@Y.T
          
        for i in range(self._num_sims):
            S = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            knn = neigh.kneighbors(1-S, return_distance=False)
            S1 = np.zeros(S.shape)
            for j in range(S.shape[0]):
                jj = knn[j]
                S1[j, jj] = S[j, jj]
            w[i] = self._alignment(S1,S_ideal)/(n*np.sqrt(self._alignment(S1,S1)))
        w = w/np.sum(w)
        return w
#----------------------------------------------------------------------------------------     
        
    def _alignment(self, S1, S2):
        A = S1*S2
        a = A.sum()
        return a     
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------   
    
        
class Combine_Sims_Limb(Combine_Sims_Ave):
    """ 
    the weight is proprotional to the 1-local imbalance
    """
    def __init__(self, k=5):
        super().__init__()
        self.k = k
        self.copyable_attrs.append('k')    
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        w = np.zeros(self._num_sims, dtype=float) 
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            w[i] = 1-self._cal_limb(S1, Y, self.k)
        w = w/np.sum(w)
        return w 
#----------------------------------------------------------------------------------------         
        
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        knns = neigh.kneighbors(1 - S, return_distance=False)
        
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            for j in range(Y.shape[1]):
                if Y[i,j] == 1: # only consider "1" 
                    C[i,j] = k-np.sum(Y[ii,j])
        C = C/k
        milb = np.sum(C)/np.sum(Y)
        
        return milb
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------
        
    
class Combine_Sims_Limb2(Combine_Sims_Limb):
    """ 
    Difference with Combine_Sims_Limb
    the weight is proprotional to the 1/(local imbalance)
    """
    def __init__(self, k=5):
        super().__init__(k)
#---------------------------------------------------------------------------------------- 
        
    def _compute_weights(self, Ss, Y):
        w = np.zeros(self._num_sims, dtype=float) 
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            w[i] = 1/self._cal_limb(S1, Y, self.k)
        w = w/np.sum(w)
        return w 
#----------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------


class Combine_Sims_Limb3(Combine_Sims_Limb):
    """ 
    Difference with Combine_Sims_Limb
    considering the influence of similarities
    !!!!!! This is one finally used in the paper
    """
    def __init__(self, k=5):
        super().__init__(k)
        
#---------------------------------------------------------------------------------------- 
    
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        idx = np.where(S>1.0)
        knns = neigh.kneighbors(1 - S, return_distance=False)
                
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            s = S[i,ii]
            z = np.sum(s)
            if z == 0:
                z=1
            C[i] = 1-s@Y[ii,:]/z
        C *= Y
        milb = np.sum(C)/np.sum(Y)
        
        return milb
#---------------------------------------------------------------------------------------- 
#----------------------------------------------------------------------------------------
        

class Combine_Sims_LowestLimb(Combine_Sims_Limb):
    """ 
    choose the one similarity with lowerest local imbalance
    """
    def __init__(self, k=5):
        super().__init__(k)
#---------------------------------------------------------------------------------------- 
    
    def _compute_weights(self, Ss, Y):
        w = np.zeros(self._num_sims, dtype=float) 
        limb_ds = np.ones(self._num_sims, dtype=float)
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            limb_ds[i] = self._cal_limb(S1, Y, self.k)
        min_idx = np.argmin(limb_ds)
        w[min_idx] = 1.0
        # w = w/np.sum(w)
        return w 
#----------------------------------------------------------------------------------------

class Combine_Sims_Single(Combine_Sims_Ave):
    """ 
    Only use one single similarity
    """
    def __init__(self, ind=0):
        super().__init__()
        self.ind = ind # ind similarity is used, others' weights are set as 0
        self.copyable_attrs.append('ind')
#---------------------------------------------------------------------------------------- 
    
    def _compute_weights(self, Ss, Y):
        w = np.zeros(self._num_sims, dtype=float) 
        w[self.ind] = 1.0
        return w 
#----------------------------------------------------------------------------------------

#----------------------------------------------------------------------------------------

In [87]:
com_KA = Combine_Sims_KA()
S1 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_ddi.txt")
S2 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_disease.txt")
S3 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_se.txt")
S4 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_tanimoto.txt")
Y_ = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt").T
similarity_matrices = np.array([S1, S2, S3, S4])

In [88]:
com_KA._compute_weights(similarity_matrices,Y_)

array([0.35971064, 0.18774675, 0.22692613, 0.22561648])

In [90]:
S_com_KA_d = 0.35971064*S1+0.18774675*S2+0.22692613*S3+0.22561648*S4


np.savetxt('S_com_KA_d.txt', S_com_KA_d, delimiter='\t')

In [91]:
S2_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_disease.txt")
S1_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_ppi.txt")
S3_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_sw-n.txt")

Y_1 = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p])


In [92]:
com_KA._compute_weights(similarity_matrices_p,Y_1)

array([0.67541219, 0.1758271 , 0.14876071])

In [93]:
S_com_KA_p = 0.67541219*S2_p+0.1758271 *S1_p+0.14876071*S3_p


np.savetxt('S_com_KA_p.txt', S_com_KA_p, delimiter='\t')